In [ ]:
"""TL_plotting_and_manual_stitching.ipynb
James Gardner
January 2021
"""

import numpy as np
import matplotlib.pyplot as plt
# import pandas as pd

In [ ]:
PATH_TO_DATA = "../data/"

# contant font sizes for plotting
SMALL_SIZE = 12
MEDIUM_SIZE = 16
BIGGER_SIZE = 22

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
def load_and_stitch(filename_tags, path_to_dir = "../data/",
                    filename_header = "SR785_23-12-2020_", plot_true=True, show_true=True, plot_label="",
                    return_data=False, show_bands=False):
    """given a list of timestamps, plots the stitched data np array for the full spectrum"""
    # numpy is clever enough to cut the metadata header from each
    data_set = np.array([np.genfromtxt(path_to_dir + filename_header + tag + ".txt") for tag in filename_tags])
    max_freqs = np.array([data[-1,0] for data in data_set])

    # sort in ascending order of max freq
    data_set = data_set[np.argsort(max_freqs)]
    max_freqs = np.sort(max_freqs)

    # stitching
    data_stitched = np.append(data_set[0],
                              np.concatenate([data[data[:,0].searchsorted(max_freqs[i]):]
                                              for i, data in enumerate(data_set[1:])]),
                              axis=0)

    # plotting
    if plot_true:
        yrange = (data_stitched[:,1].min(), data_stitched[:,1].max())

        fig, ax = plt.subplots(figsize=(12,6))
        ax.loglog(data_stitched[:,0], data_stitched[:,1], label=plot_label)
        if show_bands:
            ax.vlines(max_freqs[:-1], yrange[0], yrange[1],
                      colors=['r'], linestyles=['--'], label='frequency bands')
        ax.set(title="", ylabel=r"absolute value of FFT / $\dfrac{V_{rms}}{\sqrt{Hz}}$", xlabel="frequency / Hz",
               xlim=(None, data_stitched[:,0].max()))
        ax.legend()
        ax.grid(which="both")
        fig.savefig("TL_residual_error_full_spectrum_{0}.pdf".format(filename_tags[0]), bbox_inches="tight")
        if show_true:
            plt.show(fig)
        
    if return_data:
        return max_freqs, data_stitched

In [ ]:
# load_and_stitch(["105934", "110340", "110444", "110640"], plot_label="TL residual error")
# load_and_stitch(["111604", "112046", "112135", "112233"], plot_label="TL dark noise")
# load_and_stitch(["113737", "113841", "113923", "114113"], plot_label="TL jitter + dark noise")

In [ ]:
# currently don't have the functionality to overlay the plots, so I do that manually
max_freqs, data1 = load_and_stitch(["105934", "110340", "110444", "110640"], plot_true=False, return_data=True)
max_freqs, data2 = load_and_stitch(["111604", "112046", "112135", "112233"], plot_true=False, return_data=True)
max_freqs, data3 = load_and_stitch(["113737", "113841", "113923", "114113"], plot_true=False, return_data=True)

yrange = (min((data[:,1].min() for data in [data1, data2, data3])),
          max((data[:,1].max() for data in [data1, data2, data3])))

fig, ax = plt.subplots(figsize=(12,6))
ax.loglog(data1[:,0], data1[:,1], label="TL residual error")
ax.loglog(data2[:,0], data2[:,1], label="TL dark noise")
ax.loglog(data3[:,0], data3[:,1], label="TL jitter + dark noise")
# ax.vlines(max_freqs[:-1], yrange[0], yrange[1], colors=['r'], linestyles=['--'], label='frequency bands')
ax.set(title="", ylabel=r"absolute value of FFT / $\dfrac{V_{rms}}{\sqrt{Hz}}$",
       xlabel="frequency / Hz",
       xlim=(None, data1[:,0].max()))
ax.legend(title="2020-12-23")
ax.grid(which="both")
fig.savefig("TL_residual_error_full_spectrum_comparison_no_bands.pdf", bbox_inches="tight")
fig.savefig("TL_residual_error_full_spectrum_comparison_no_bands.jpg", bbox_inches="tight")
plt.show(fig)

In [ ]:
# we want the gradient at the zero crossing near resonance
# here, I do this by eye, sadly
PATH_TO_DATA = "../data/"

scope_data = np.genfromtxt(PATH_TO_DATA+"2021-01-06_scope_1.csv", delimiter=",", skip_header=2, skip_footer=8)

scope_time, scope_xdiff, scope_ydiff = scope_data[:,0], scope_data[:,2], scope_data[:,4]

t0, t1 = 0.0051, 0.01
it0, it1 = scope_time.searchsorted(t0), scope_time.searchsorted(t1)
scope_time_cut = scope_time[it0:it1]
scope_xdiff_cut = scope_xdiff[it0:it1]
scope_ydiff_cut = scope_ydiff[it0:it1]

fig, ax = plt.subplots(figsize=(12, 8))
ax.plot(scope_time_cut, scope_xdiff_cut, label="XDIFF", color="darkred")
ax.plot(scope_time_cut, scope_ydiff_cut, label="YDIFF", color="darkblue")

# line of best fit at zero crossing
point_range = 100
xdiff_zero_xing = np.argmax(scope_xdiff_cut<0)
xing0, xing1 = xdiff_zero_xing - point_range, xdiff_zero_xing + point_range
xing_cut_time, xing_cut_xdiff = scope_time_cut[xing0:xing1], scope_xdiff_cut[xing0:xing1]
xdiff_a1, xdiff_a0 = np.polyfit(xing_cut_time, xing_cut_xdiff, 1)
ax.plot(scope_time_cut, np.poly1d([xdiff_a1, xdiff_a0])(scope_time_cut),
        label="XDIFF fit: {0:.4g} t + {1:.4g}".format(xdiff_a1, xdiff_a0), color="orange")

ydiff_zero_xing = np.argmax(scope_ydiff_cut>0)
# looks better if 50 crossing
# ydiff_zero_xing = np.argmax(scope_ydiff_cut>50)
xing0, xing1 = ydiff_zero_xing - point_range, ydiff_zero_xing + point_range
xing_cut_time, xing_cut_ydiff = scope_time_cut[xing0:xing1], scope_ydiff_cut[xing0:xing1]
ydiff_a1, ydiff_a0 = np.polyfit(xing_cut_time, xing_cut_ydiff, 1)
ax.plot(scope_time_cut, np.poly1d([ydiff_a1, ydiff_a0])(scope_time_cut),
        label="YDIFF fit: {0:.4g} t + {1:.4g}".format(ydiff_a1, ydiff_a0), color="lime")

ax.grid()
ax.set(xlabel="time, t / s", ylabel="voltage / V", xlim=(t0, t1), ylim=(-0.2, 0.3))
ax.legend(title="2021-01-06")
fig.savefig("TL_YDIFF_and_XDIFF_error_signal_timeseries.pdf")
fig.savefig("TL_YDIFF_and_XDIFF_error_signal_timeseries.jpg")
plt.show()

In [ ]:
max_freqs, data4 = load_and_stitch(["112751", "112851", "113003", "113244"], filename_header="SR785_06-01-2021_",
                                   plot_true=False, return_data=True)
max_freqs, data5 = load_and_stitch(["114129", "114229", "114325", "114550"], filename_header="SR785_06-01-2021_",
                                   plot_true=False, return_data=True)

# need to normalise change in power per change in cavity length using oscilloscope data from scanning the cavity
zero_crossing_normalisation_factor = abs(ydiff_a1/xdiff_a1)
ydiff_resi_power = data5[:,1]
xdiff_resi_power = zero_crossing_normalisation_factor*data5[:,2]
print("zero crossing normalisation factor: {0:.4g}".format(zero_crossing_normalisation_factor))

yrange = (min((data[:,1].min() for data in [data4, data5])),
          max((data[:,1].max() for data in [data4, data5])))

fig, ax = plt.subplots(figsize=(12,6))
ax.loglog(data4[:,0], data4[:,1], label="YDIFF - dark noise")
ax.loglog(data4[:,0], data4[:,2], label="XDIFF - dark noise")
ax.loglog(data5[:,0], ydiff_resi_power, label="YDIFF - residual error")
ax.loglog(data5[:,0], xdiff_resi_power, label="XDIFF - residual error")
# ax.vlines(max_freqs[:-1], yrange[0], yrange[1], colors=['r'], linestyles=['--'], label='frequency bands')

ax.set(title="", ylabel=r"absolute value of FFT / $\dfrac{V_{rms}}{\sqrt{Hz}}$",
       xlabel="frequency / Hz",
       xlim=(None, data4[:,0].max()))
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1], title="2021-01-06")
ax.grid(which="both")
filename = "TL_XDIFF_and_YDIFF_spectra_comparison"
fig.savefig(filename+".pdf", bbox_inches="tight")
fig.savefig(filename+".jpg", bbox_inches="tight")
plt.show(fig)

In [ ]:
# this variation in days was likely simply due to previously using the servo monitor port which has a ~100 gain
fig, ax = plt.subplots(figsize=(12,6))
ax.loglog(data1[:,0], data1[:,1], label="2020-12-23: YDIFF - residual error")
ax.loglog(data2[:,0], data2[:,1], label="2020-12-23: YDIFF - dark noise")
ax.loglog(data4[:,0], data4[:,1], label="2021-01-06: YDIFF - dark noise")
ax.loglog(data5[:,0], data5[:,1], label="2021-01-06: YDIFF - residual error")

ax.set(title="", ylabel=r"absolute value of FFT / $\dfrac{V_{rms}}{\sqrt{Hz}}$", xlabel="frequency / Hz",
       xlim=(None, data4[:,0].max()))
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1])
ax.grid(which="both")
filename = "TL_YDIFF_comparing_days"
fig.savefig(filename+".pdf", bbox_inches="tight")
fig.savefig(filename+".jpg", bbox_inches="tight")
plt.show(fig)

In [ ]:
# length noise: view the scan as over one wavelength and use that to calibrate volts to metres

# finding the fundamental resonances (or rather, the spacing by finding the same feature: the minimum)
j0, j1 = scope_ydiff.argmin(), len(scope_ydiff)//2 + scope_ydiff[len(scope_ydiff)//2:].argmin()
# plt.plot(scope_time[j0:j1], scope_ydiff[j0:j1])
time_scale = scope_time[j1] - scope_time[j0]
# wavelength is 532 nm
length_scale = 532e-9
scope_length = length_scale/time_scale*scope_time

scope_length_cut = scope_length[it0:it1]

fig, ax = plt.subplots(figsize=(12, 8))
ax.plot(scope_length_cut, scope_xdiff_cut, label="XDIFF", color="darkred")
ax.plot(scope_length_cut, scope_ydiff_cut, label="YDIFF", color="darkblue")

# use chain rule to convert slopes to V/m
xdiff_a1_length, ydiff_a1_length = time_scale/length_scale*xdiff_a1, time_scale/length_scale*ydiff_a1
ax.plot(scope_length_cut, np.poly1d([xdiff_a1_length, xdiff_a0])(scope_length_cut),
        label="XDIFF fit: {0:.4g} d + {1:.4g}".format(xdiff_a1_length, xdiff_a0), color="orange")
ax.plot(scope_length_cut, np.poly1d([ydiff_a1_length, ydiff_a0])(scope_length_cut),
        label="YDIFF fit: {0:.4g} d + {1:.4g}".format(ydiff_a1_length, ydiff_a0), color="lime")

ax.grid()
ax.set(xlabel="length, d / m", ylabel="voltage / V",
       xlim=(length_scale/time_scale*t0, length_scale/time_scale*t1), ylim=(-0.2, 0.3))
ax.legend(title="2021-01-06")
fig.savefig("TL_YDIFF_and_XDIFF_length_scan.pdf")
fig.savefig("TL_YDIFF_and_XDIFF_length_scan.jpg")
plt.show()

In [ ]:
# calibrating Vrms/rtHz to m/rtHz and normalising by above length slopes (in V/m)

ydiff_resi_calib = data5[:,1]/abs(ydiff_a1_length)
xdiff_resi_calib = data5[:,2]/abs(xdiff_a1_length)

yrange = (min((data[:,1].min() for data in [data4, data5])),
          max((data[:,1].max() for data in [data4, data5])))

fig, ax = plt.subplots(figsize=(12,6))
# ax.loglog(data4[:,0], data4[:,1], label="YDIFF - dark noise")
# ax.loglog(data4[:,0], data4[:,2], label="XDIFF - dark noise")
ax.loglog(data5[:,0], ydiff_resi_calib, label="YDIFF - residual error - length noise", color="g")
ax.loglog(data5[:,0], xdiff_resi_calib, label="XDIFF - residual error - length noise", color="r")
# ax.vlines(max_freqs[:-1], yrange[0], yrange[1], colors=['r'], linestyles=['--'], label='frequency bands')

ax.set(title="", ylabel=r"abs(FFT/slope at zero crossing) / $\dfrac{m}{\sqrt{Hz}}$",
       xlabel="frequency / Hz",
       xlim=(None, data4[:,0].max()))
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1], title="2021-01-06")
ax.grid(which="both")
filename = "TL_XDIFF_and_YDIFF_length_noise"
fig.savefig(filename+".pdf", bbox_inches="tight")
fig.savefig(filename+".jpg", bbox_inches="tight")
plt.show(fig)

In [ ]:
# 2021-01-13 dark noise comparison
max_freqs, dark_data = load_and_stitch(["120840", "121019", "121105", "121202"],
                                       filename_header="SR785_13-01-2021_",
                                       plot_true=False, return_data=True)
dark_freqs, dark_tl, dark_pdh = (dark_data[:,i] for i in range(3))

fig, ax = plt.subplots(figsize=(12,6))
ax.loglog(dark_freqs, dark_tl, label="TL YDIFF - dark noise")
ax.loglog(dark_freqs, dark_pdh, label="PDH - dark noise")
ax.set(title="", ylabel=r"abs(FFT) / $\dfrac{V_{rms}}{\sqrt{Hz}}$",
       xlabel="frequency / Hz",
       xlim=(None, dark_freqs.max()))
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1], title="2021-01-13")
ax.grid(which="both")
filename = "TL_dark_noise_comparison_to_PDH"
fig.savefig(filename+".pdf", bbox_inches="tight")
fig.savefig(filename+".jpg", bbox_inches="tight")
plt.show(fig)

# 2021-01-14 dark noise comparison
max_freqs, dark_data_2 = load_and_stitch(["104438","104344","104251","104102"],
                                       filename_header="SR785_14-01-2021_",
                                       plot_true=False, return_data=True)
dark_freqs_2, dark_tl_2, dark_pdh_2 = (dark_data_2[:,i] for i in range(3))

fig, ax = plt.subplots(figsize=(12,6))
ax.loglog(dark_freqs_2, dark_tl_2, label="TL YDIFF - dark noise")
ax.loglog(dark_freqs_2, dark_pdh_2, label="PDH new mixer - dark noise")
ax.set(title="", ylabel=r"abs(FFT) / $\dfrac{V_{rms}}{\sqrt{Hz}}$",
       xlabel="frequency / Hz",
       xlim=(None, dark_freqs_2.max()))
ax.legend(title="2021-01-14")
ax.grid(which="both")
filename = "TL_dark_noise_comparison_to_PDH_new_mixer"
fig.savefig(filename+".pdf", bbox_inches="tight")
fig.savefig(filename+".jpg", bbox_inches="tight")
plt.show(fig)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,6), sharey=True)
ax1.loglog(dark_freqs, dark_tl, label="TL YDIFF - dark noise")
ax1.loglog(dark_freqs, dark_pdh, label="old mixer PDH - dark noise")
ax2.loglog(dark_freqs_2, dark_tl_2, label="TL YDIFF - dark noise")
ax2.loglog(dark_freqs_2, dark_pdh_2, label="new mixer PDH - dark noise")
ax1.set(title="", ylabel=r"abs(FFT) / $\dfrac{V_{rms}}{\sqrt{Hz}}$",
       xlabel="frequency / Hz",
       xlim=(None, dark_freqs.max()))
ax2.set(title="", ylabel="",
       xlabel="frequency / Hz",
       xlim=(None, dark_freqs_2.max()))
ax1.legend(title="2021-01-13")
ax2.legend(title="2021-01-14")
ax1.grid(which="both")
ax2.grid(which="both")
fig.subplots_adjust(wspace=0.02)
filename = "TL_dark_noise_comparison_to_PDH_two_panels"
fig.savefig(filename+".pdf", bbox_inches="tight")
fig.savefig(filename+".jpg", bbox_inches="tight")
plt.show(fig)

In [ ]:
# 2021-01-13 TL and PDH same peak-to-peak
# PATH_TO_DATA = "../data/"
filename = "scope_2021-01-13_error_signals.csv"
scope_data = np.genfromtxt(PATH_TO_DATA+filename, delimiter=",", skip_header=41, skip_footer=0)

# time, QPD_sum, PDH_err, ramp_scan, QPD_YDIFF
scope_time, scope_pdh, scope_tl = scope_data[:,0], scope_data[:,2], scope_data[:,4]

# cut to around a resonance
t0, t1 = 0.0255, 0.0285
it0, it1 = scope_time.searchsorted(t0), scope_time.searchsorted(t1)
scope_time_cut = scope_time[it0:it1]
scope_pdh_cut = scope_pdh[it0:it1]
scope_tl_cut = scope_tl[it0:it1]

fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(scope_time_cut, scope_pdh_cut, label="PDH error signal")
ax.plot(scope_time_cut, scope_tl_cut, label="TL error signal")

# line of best fit at zero crossing
point_range = 100
pdh_zero_xing = np.argmax(scope_pdh_cut>0)
xing0, xing1 = pdh_zero_xing - point_range, pdh_zero_xing + point_range
xing_cut_time, xing_cut_pdh = scope_time_cut[xing0:xing1], scope_pdh_cut[xing0:xing1]
pdh_a1, pdh_a0 = np.polyfit(xing_cut_time, xing_cut_pdh, 1)
ax.plot(scope_time_cut, np.poly1d([pdh_a1, pdh_a0])(scope_time_cut),
        label="PDH fit: {0:.4g} t + {1:.4g}".format(pdh_a1, pdh_a0))

tl_zero_xing = np.argmax(scope_tl_cut>0)
xing0, xing1 = tl_zero_xing - point_range, tl_zero_xing + point_range
xing_cut_time, xing_cut_tl = scope_time_cut[xing0:xing1], scope_tl_cut[xing0:xing1]
tl_a1, tl_a0 = np.polyfit(xing_cut_time, xing_cut_tl, 1)
ax.plot(scope_time_cut, np.poly1d([tl_a1, tl_a0])(scope_time_cut),
        label="TL fit: {0:.4g} t + {1:.4g}".format(tl_a1, tl_a0))

ax.grid()
ax.set(xlabel="time, t / s", ylabel="voltage / V", xlim=(t0, t1), ylim=(-0.15, 0.15))
ax.legend(title="2021-01-13")
fig.savefig("TL_and_PDH_error_signal_timeseries.pdf")
fig.savefig("TL_and_PDH_error_signal_timeseries.jpg")
plt.show()

# length noise: view the scan as over one wavelength and use that to calibrate volts to metres
# finding the fundamental resonances (or rather, the spacing by finding the same feature: the minimum)
j0, j1 = scope_tl.argmin(), len(scope_tl)//2 + scope_tl[len(scope_tl)//2:].argmin()
# plt.plot(scope_time[j0:j1], scope_tl[j0:j1])
time_scale = scope_time[j1] - scope_time[j0]
# wavelength is 532 nm
length_scale = 532e-9
scope_length = length_scale/time_scale*scope_time

scope_length_cut = scope_length[it0:it1]

fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(scope_length_cut, scope_pdh_cut, label="PDH error signal")
ax.plot(scope_length_cut, scope_tl_cut, label="TL error signal")

# use chain rule to convert slopes to V/m
pdh_a1_length, tl_a1_length = time_scale/length_scale*pdh_a1, time_scale/length_scale*tl_a1
ax.plot(scope_length_cut, np.poly1d([pdh_a1_length, pdh_a0])(scope_length_cut),
        label="PDH fit: {0:.4g} d + {1:.4g}".format(pdh_a1_length, pdh_a0))
ax.plot(scope_length_cut, np.poly1d([tl_a1_length, tl_a0])(scope_length_cut),
        label="TL fit: {0:.4g} d + {1:.4g}".format(tl_a1_length, tl_a0))

ax.grid()
ax.set(xlabel="length, d / m", ylabel="voltage / V",
       xlim=(length_scale/time_scale*t0, length_scale/time_scale*t1), ylim=(-0.15, 0.15))
ax.legend(title="2021-01-13")
fig.savefig("TL_and_PDH_length_scan.pdf")
fig.savefig("TL_and_PDH_length_scan.jpg")
plt.show()

In [ ]:
# 2021-01-13 residual error and dark noise
# freq, TL YDIFF, PDH
weird_dark = np.genfromtxt(PATH_TO_DATA+"SR785_13-01-2021_114258.txt")
weird_residual = np.genfromtxt(PATH_TO_DATA+"SR785_13-01-2021_114612.txt")

fig, ax = plt.subplots(figsize=(12,6))
ax.loglog(weird_dark[:,0], weird_dark[:,1], label="TL - dark noise")
ax.loglog(weird_dark[:,0], weird_dark[:,2], label="PDH - dark noise")
ax.loglog(weird_residual[:,0], weird_residual[:,1], label="TL - residual error under TL")
ax.loglog(weird_residual[:,0], weird_residual[:,2], label="PDH - residual error under TL")

ax.set(title="", ylabel=r"abs(FFT) / $\dfrac{V_{rms}}{\sqrt{Hz}}$",
       xlabel="frequency / Hz",
       xlim=(None, weird_dark[:,0].max()))
ax.grid(which="both")
ax.legend(title="2021-01-13")
filename = "TL_and_PDH_comparison_to_dark_noise_2021-01-13"
fig.savefig(filename+".pdf", bbox_inches="tight")
fig.savefig(filename+".jpg", bbox_inches="tight")
plt.show()

In [ ]:
# 2021-01-14 TL'd residual error against various gain values, TL and PDH (new mixer)
filename_header = "SR785_14-01-2021_"
filetag_dict = {110554:3.64/62e-3, 111835:680e-3/62e-3, 112526:1.37/62e-3,
                113243:2.41/62e-3, 113707:3.70/62e-3, 114336: 160e-3/62e-3}

fig, (ax0, ax1) = plt.subplots(2, 1, sharex=True, figsize=(12,12))

data = np.genfromtxt(PATH_TO_DATA+filename_header+repr(104102)+".txt")
ax0.loglog(data[:,0], data[:,1], label="TL dark noise", linewidth=3)
ax1.loglog(data[:,0], data[:,2], label="PDH dark noise", linewidth=3)

for filetag, gain in sorted(filetag_dict.items(), key=lambda item: item[1]):
    data = np.genfromtxt(PATH_TO_DATA+filename_header+repr(filetag)+".txt")
    ax0.loglog(data[:,0], data[:,1], label="TL, gain = {0:.3g}".format(gain))
    ax1.loglog(data[:,0], data[:,2], label="PDH, gain = {0:.3g}".format(gain))

ax0.set(title="", ylabel=r"abs(FFT) / $\dfrac{V_{rms}}{\sqrt{Hz}}$")
ax0.legend(title="2021-01-14")
ax0.grid(which="both")
ax1.set(ylabel=r"abs(FFT) / $\dfrac{V_{rms}}{\sqrt{Hz}}$",
       xlabel="frequency / Hz",
       xlim=(None, data[:,0].max()))
ax1.legend(title="2021-01-14")
ax1.grid(which="both")
fig.subplots_adjust(hspace=0.03)
filename = "TL_and_PDH_TLd_residual_error_different_gains"
fig.savefig(filename+".pdf", bbox_inches="tight")
fig.savefig(filename+".jpg", bbox_inches="tight")
plt.show(fig)

In [ ]:
# project TL dark noise and PDH high gain noise floor into length noise and compare

# TL, PDH dark noise, from 2021-01-18
dark_data = np.genfromtxt(PATH_TO_DATA + "SR785_18-01-2021_133426.txt")
dark_freq, dark_tl, dark_pdh = (dark_data[:,i] for i in range(3))
# TL, PDH jitter noise, from 2021-01-18
jitter_data = np.genfromtxt(PATH_TO_DATA + "SR785_18-01-2021_134107.txt")
jitter_freq, jitter_tl, jitter_pdh = (jitter_data[:,i] for i in range(3))
# PDH high gain noise floor, gain = 58.7, from 2021-01-14
gain_data = np.genfromtxt(PATH_TO_DATA + "SR785_14-01-2021_110554.txt")
gain_freq, gain_pdh = (gain_data[:,i] for i in (0, 2))

# project/calibrate to length noise using 2021-01-13 data
# pdh_a1_length is slope in V/m
(dark_tl_length, dark_pdh_length, jitter_tl_length, jitter_pdh_length,
 gain_pdh_length) = (dark_tl/tl_a1_length, dark_pdh/pdh_a1_length,
                     jitter_tl/tl_a1_length, jitter_pdh/pdh_a1_length, gain_pdh/pdh_a1_length)

# calibrate jitter and resi. error measurements to DC voltage from QPD SUM 
dc_voltage_2021_01_18 = 312.5 #mV
# plt.plot(scope_time, scope_data[:,1])
dc_voltage_2021_01_14 = 181 #mV
dc_voltage_correction = dc_voltage_2021_01_18/dc_voltage_2021_01_14
print("""voltage on resonance: {0:.3g} V, off resonance: {1:.3g} V,
correction factor = {2:.3g}""".format(scope_data[:,1].min(), scope_data[:,1].max(), dc_voltage_correction))
gain_pdh_length_corrected = gain_pdh_length*dc_voltage_correction

fig, ax = plt.subplots(figsize=(12,6))
ax.loglog(dark_freq, dark_tl_length, label="TL dark length noise")
ax.loglog(dark_freq, dark_pdh_length, label="PDH dark length noise")
ax.loglog(jitter_freq, jitter_tl_length, label="TL jitter length noise")
ax.loglog(jitter_freq, jitter_pdh_length, label="PDH jitter length noise")
ax.loglog(gain_freq, gain_pdh_length_corrected, label="""PDH residual error (length noise floor),
servo gain 58.7, corrected for DC voltage""")
ax.set(title="", ylabel=r"abs(FFT)/slope / $\dfrac{m_{rms}}{\sqrt{Hz}}$",
       xlabel="frequency / Hz",
       xlim=(None, dark_freq.max()))
ax.legend(title="2021-01-18 dark and jitter noise\n2021-01-14 PDH residual error\n2021-01-13 slope calibration")
ax.grid(which="both")
filename = "TL_length_noise_compared_to_PDH_length_noise_floor"
fig.savefig(filename+".pdf", bbox_inches="tight")
fig.savefig(filename+".jpg", bbox_inches="tight")
plt.show(fig)

In [ ]:
# 2021-01-20 TL and PDH
filename = "2021-01-20_scope_2.csv"
scope_data = np.genfromtxt(PATH_TO_DATA + filename, delimiter=",", skip_header=3, skip_footer=0)

# time, QPD_sum, PDH_err, ramp_scan, QPD_YDIFF
scope_time, scope_pdh, scope_tl = scope_data[:,0], scope_data[:,2], scope_data[:,4]

# length noise: view the scan as over one wavelength and use that to calibrate volts to metres
# finding the fundamental resonances (or rather, the spacing by finding the same feature: the minimum)
# note: it is critical that the data shows two resonances and only two
j0, j1 = scope_pdh.argmin(), len(scope_pdh)//2 + scope_pdh[len(scope_pdh)//2:].argmin()
time_scale = scope_time[j1] - scope_time[j0]
# wavelength is 532 nm
length_scale = 532e-9
time_to_length_conversion = length_scale/time_scale
scope_length = time_to_length_conversion*scope_time

# cut to around a resonance, need to find manually right now
# to-do, use argmin and argmax to find the resonance automatically
l0, l1 = 4.2e-7, 5.3e-7
il0, il1 = scope_length.searchsorted(l0), scope_length.searchsorted(l1)
scope_length_cut = scope_length[il0:il1]
scope_pdh_cut = scope_pdh[il0:il1]
scope_tl_cut = scope_tl[il0:il1]

# line of best fit at zero crossing
# have lowered the point_range due to a drop in sampling rate, cause unknown
point_range = 10 
pdh_zero_xing = np.argmax(scope_pdh_cut>0)
xing0, xing1 = pdh_zero_xing - point_range, pdh_zero_xing + point_range
xing_cut_length, xing_cut_pdh = scope_length_cut[xing0:xing1], scope_pdh_cut[xing0:xing1]
pdh_a1_length, pdh_a0 = np.polyfit(xing_cut_length, xing_cut_pdh, 1)

tl_zero_xing = np.argmax(scope_tl_cut>0)
xing0, xing1 = tl_zero_xing - point_range, tl_zero_xing + point_range
xing_cut_length, xing_cut_tl = scope_length_cut[xing0:xing1], scope_tl_cut[xing0:xing1]
tl_a1_length, tl_a0 = np.polyfit(xing_cut_length, xing_cut_tl, 1)

# plotting
fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(scope_length_cut, scope_pdh_cut, label="PDH error signal")
ax.plot(scope_length_cut, scope_tl_cut, label="TL error signal")
ax.plot(scope_length_cut, np.poly1d([pdh_a1_length, pdh_a0])(scope_length_cut),
        label="PDH fit: {0:.4g} d + {1:.4g}".format(pdh_a1_length, pdh_a0))
ax.plot(scope_length_cut, np.poly1d([tl_a1_length, tl_a0])(scope_length_cut),
        label="TL fit: {0:.4g} d + {1:.4g}".format(tl_a1_length, tl_a0))
ax.grid()
ax.set(xlabel="length, d / m", ylabel="voltage / V",
       xlim=(l0, l1), ylim=(scope_pdh_cut.min()-0.02, scope_pdh_cut.max()+0.02))
ax.legend(title="2021-01-20")
fig.savefig("TL_and_PDH_length_scan_2021-01-20.pdf")
fig.savefig("TL_and_PDH_length_scan_2021-01-20.jpg")
plt.show()

In [ ]:
# 2021-01-20, re-doing gain measurements

def SR785_length_signals(file_tag, tl_slope, pdh_slope, file_header="SR785_20-01-2021_", return_freq=True):
    """loads a SR785 file and returns the freq scale and the TL and PDH length noises
    requires the calibration slopes from the error signals
    """
    data = np.genfromtxt(PATH_TO_DATA + file_header + file_tag + ".txt")
    freq, tl, pdh = (data[:,i] for i in range(3))
    tl_length, pdh_length = tl/tl_slope, pdh/pdh_slope
    
    if return_freq:
        return freq, tl_length, pdh_length
    else:
        return tl_length, pdh_length

file_header = "SR785_20-01-2021_"
# dark noise
file_tag = "104718"
dark_freq, dark_tl_length, dark_pdh_length = SR785_length_signals(file_tag, tl_a1_length, pdh_a1_length)

# jitter noise
file_tag = "103109"
jitter_tl_length, jitter_pdh_length = SR785_length_signals(file_tag, tl_a1_length, pdh_a1_length,
                                                           return_freq=False)

# residual errors under TL, high gain
file_tag = "110840"
high_gain_tl_length, high_gain_pdh_length = SR785_length_signals(file_tag, tl_a1_length, pdh_a1_length,
                                                                 return_freq=False)

# residual errors under TL, low gain
file_tag = "111444"
low_gain_tl_length, low_gain_pdh_length = SR785_length_signals(file_tag, tl_a1_length, pdh_a1_length,
                                                               return_freq=False)

# DC offset for normalisation
# PDH resi error
DC_power_resonance = scope_data[:,1].min()
# jitter noise
filename = "2021-01-20_scope_0.csv"
scope_0_data = np.genfromtxt(PATH_TO_DATA + filename, delimiter=",", skip_header=10, skip_footer=0)
DC_power_jitter = scope_0_data[:,1].mean()
DC_power_correction = DC_power_jitter/DC_power_resonance

high_gain_pdh_length_corrected, low_gain_pdh_length_corrected = (high_gain_pdh_length*DC_power_correction,
                                                                 low_gain_pdh_length*DC_power_correction)
high_gain_tl_length_corrected, low_gain_tl_length_corrected = (high_gain_tl_length*DC_power_correction,
                                                               low_gain_tl_length*DC_power_correction)

# plotting
fig, ax = plt.subplots(figsize=(12,6))
ax.loglog(dark_freq, dark_tl_length, label="TL dark length noise")
ax.loglog(dark_freq, dark_pdh_length, '--', label="PDH dark length noise")
ax.loglog(dark_freq, jitter_tl_length, label="TL jitter length noise")
ax.loglog(dark_freq, jitter_pdh_length, '--', label="PDH jitter length noise")
ax.loglog(dark_freq, high_gain_pdh_length_corrected, '--',
          label="PDH residual error,\nservo gain 67.1 (high)")
ax.loglog(dark_freq, low_gain_pdh_length_corrected, '--',
          label="PDH residual error,\nservo gain 2.67 (low)")
ax.loglog(dark_freq, high_gain_tl_length_corrected,
          label="TL residual error,\nservo gain 67.1 (high)")
ax.loglog(dark_freq, low_gain_tl_length_corrected,
          label="TL residual error,\nservo gain 2.67 (low)")
ax.set(title="", ylabel=r"abs(FFT)/slope / $\dfrac{m_{rms}}{\sqrt{Hz}}$",
       xlabel="frequency / Hz",
       xlim=(None, dark_freq.max()))
handles, labels = ax.get_legend_handles_labels()
legend_order = (5,7,4,2,0,6,1,3)
handles_new, labels_new = [handles[i] for i in legend_order], [labels[i] for i in legend_order]
ax.legend(handles_new, labels_new, title="2021-01-20")
ax.grid(which="both")
filename = "TL_and_PDH_TLd_residual_errors_two_gains"
fig.savefig(filename+".pdf", bbox_inches="tight")
fig.savefig(filename+".jpg", bbox_inches="tight")
plt.show(fig)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,6), sharey=True)
ax1.loglog(dark_freq, dark_tl_length, label="TL dark length noise")
ax2.loglog(dark_freq, dark_pdh_length, label="PDH dark length noise")
ax1.loglog(dark_freq, jitter_tl_length, label="TL jitter length noise")
ax2.loglog(dark_freq, jitter_pdh_length, label="PDH jitter length noise")
ax2.loglog(dark_freq, high_gain_pdh_length_corrected,
           label="PDH residual error,\nservo gain 67.1 (high)")
ax2.loglog(dark_freq, low_gain_pdh_length_corrected,
          label="PDH residual error,\nservo gain 2.67 (low)")
ax1.loglog(dark_freq, high_gain_tl_length_corrected,
          label="TL residual error,\nservo gain 67.1 (high)")
ax1.loglog(dark_freq, low_gain_tl_length_corrected,
          label="TL residual error,\nservo gain 2.67 (low)")
ax2.loglog(dark_freq, jitter_tl_length, '--', label="projected TL jitter length noise")
ax1.set(title="", ylabel=r"abs(FFT)/slope / $\dfrac{m_{rms}}{\sqrt{Hz}}$",
       xlabel="frequency / Hz",
       xlim=(None, dark_freq.max()))
ax1.legend(title="2021-01-20")
ax2.set(title="", ylabel="",
       xlabel="frequency / Hz",
       xlim=(None, dark_freq.max()))
handles, labels = ax1.get_legend_handles_labels()
legend_order = (3,1,0,2)
ax1.legend([handles[i] for i in legend_order], [labels[i] for i in legend_order], title="2021-01-20")
handles, labels = ax2.get_legend_handles_labels()
legend_order = (3,2,4,0,1)
ax2.legend([handles[i] for i in legend_order], [labels[i] for i in legend_order], title="2021-01-20")
ax1.grid(which="both")
ax2.grid(which="both")
fig.subplots_adjust(wspace=0.02)
filename = "TL_and_PDH_TLd_residual_errors_two_gains_two_panels"
fig.savefig(filename+".pdf", bbox_inches="tight")
fig.savefig(filename+".jpg", bbox_inches="tight")
plt.show(fig)